In [8]:
import pandas as pd
from datetime import timedelta


In [9]:
medicamentos=pd.read_excel("../Bases limpias/medicamentos_19_jun_AM.xlsx")
medicamentos_IDtrain=pd.read_excel("Lista_pacientes_train.xlsx")


In [10]:
## esta función toma un registro de medicamento y regresa la marcación con respecto a 
## adherencia, si es una continuación del mismo medicamento retorna un dataframe de pandas 
## de una sola fila

def marca_fila(ID_PACIENTE,FECHA_EMISION,NombreAdherencia,
               NUMERO_CANTIDAD_PRESTACIONES,base_med=medicamentos,
              columna="NOMBRE_MEDICAMENTO",tolerancia_dias=5,Frecuencia_Mensual=30):
    #los input corresponden a campos de la tabla Medicamentos, Frecuencia_Diaria es la cantidad 
    #de ese medicamento si la aplicacion fuera diaria, ej si es un inhalador al mes usar 1, si es un
    # si es una aplicacion diaria usar 30
    # base med es la base donde se realiza la busqueda
    # Fecha Emision formato datetime de pandas
    _tiempo_esperado_=30*NUMERO_CANTIDAD_PRESTACIONES/Frecuencia_Mensual
    #_Fecha_Fin_inf_=_Fecha_Fin_-timedelta(days=tolerancia_dias)
    #_Fecha_Fin_sup_=_Fecha_Fin_+timedelta(days=tolerancia_dias)
    _filtro_fin_med_=(base_med.ID_PACIENTE==ID_PACIENTE) &(base_med["FECHA_EMISION"]>FECHA_EMISION)
    _filtro_fin_med_=_filtro_fin_med_ &(base_med[columna]==NombreAdherencia)
    _medicamentos_local_=base_med[_filtro_fin_med_]
    _fecha_prox_formula_=_medicamentos_local_.loc[base_med.ID_PACIENTE==ID_PACIENTE,"FECHA_EMISION"].min()
    _tiempo_entre_form_=(_fecha_prox_formula_-FECHA_EMISION).days
    _fecha_esperada_reclamacion_=FECHA_EMISION+_tiempo_esperado_*timedelta(days=1)
    if _medicamentos_local_.shape[0]==0:
        ## marca como no adherente 
        _adherencia_=float("NaN")
        _comentario_="ultimo registro"
        #print("No adherente")
        pass
    elif _tiempo_entre_form_<=_tiempo_esperado_+tolerancia_dias:
        ## marca como adherente marca continuidad medicamento
        #print("Adherente, Continuidad")
        _adherencia_=1
        _comentario_="adherente"
        pass
    elif _tiempo_entre_form_<2*_tiempo_esperado_:
        ## marca como aderente no continuidad del medicamento
        #print("Adherente, No Continuidad")
        _adherencia_=0
        _comentario_="no adherente"
        pass
    else:
        _adherencia_=0
        _comentario_="abandono/ciclo corto"
    #print(_medicamentos_local_)
    _return_=pd.Series({"ID_PACIENTE":ID_PACIENTE,
                           "FECHA_EMISION":FECHA_EMISION,
                        "FechaSiguienteRecla":_fecha_prox_formula_,
                           columna:NombreAdherencia,
                        "FechaEsperadaReclamacion":_fecha_esperada_reclamacion_,
                          "AdherenciaMed":_adherencia_,
                          "Comentario":_comentario_,
                       "TiempoEntreFormulas":_tiempo_entre_form_,
                        "NUMERO_CANTIDAD_PRESTACIONES":NUMERO_CANTIDAD_PRESTACIONES})
    return _return_
    
    

In [11]:
## Esta funcion toma un dataframe y añade las columnas necesarias al mismo
## oportunidad de mejora un diccionario de las prestaciones con la frecuencia 
## de medicacion
def procesa_medicamentos_df(base_med=medicamentos,columna="NOMBRE_MEDICAMENTO",NombreAdherencia="MONTELUKAST",
                           tolerancia_dias=5,Frecuencia_Mensual=30):
    _work_base_=base_med[base_med[columna]==NombreAdherencia]
    _adherencia_=_work_base_.apply(lambda x:marca_fila(x.ID_PACIENTE,
                                   x.FECHA_EMISION,
                                   NombreAdherencia,
                                   x.NUMERO_CANTIDAD_PRESTACIONES,
                                   _work_base_,
                                   columna,
                                    tolerancia_dias,Frecuencia_Mensual) ,axis=1)
    #_adherencia_=_adherencia_.loc[:,["adherencia_med","comentario"]]
    #_return_=pd.concat([df,_adherencia_],axis=1)
    return _adherencia_

In [12]:
medicamentos.Categoria.drop_duplicates()

0         OMALIZUMAB
1                NaN
4         SALBUTAMOL
241     FEXOFENADINA
427        BILASTINA
439       MOMETASONA
440      MONTELUKAST
441      FLUTICASONA
458     PREDNISOLONA
974       BUDESONIDA
1116       TIOTROPIO
2275    BENRALIZUMAB
4662     MEPOLIZUMAB
Name: Categoria, dtype: object

In [16]:

def crea_base(medicamento,Frecuencia_Mensual=30):
    _base_=procesa_medicamentos_df(base_med=medicamentos,columna="Categoria",NombreAdherencia=medicamento,Frecuencia_Mensual=Frecuencia_Mensual)
    _base_train_=_base_[_base_.ID_PACIENTE.isin(medicamentos_IDtrain.ID_PACIENTE)]
    _base_test_=_base_[~_base_.ID_PACIENTE.isin(medicamentos_IDtrain.ID_PACIENTE)]
    _base_train_.to_excel("Train_"+medicamento+".xlsx")
    _base_test_.to_excel("Test_"+medicamento+".xlsx")
    return  

In [19]:
crea_base("SALBUTAMOL",Frecuencia_Mensual=1)

In [20]:
biologicos=pd.read_excel("../Bases limpias/Biologicos_19_jun_AM.xlsx")
biologicos_IDtrain=pd.read_excel("Lista_pacientes_biologicos_train.xlsx")

In [21]:
biologicos.columns
biologicos["FECHA_EMISION"]=biologicos["FECHA_DCTO"]
biologicos['NUMERO_CANTIDAD_PRESTACIONES']=1
biologicos.NOM_GENERICO.drop_duplicates()

0        OMALIZUMAB
139    BENRALIZUMAB
150       DUPILUMAB
444     MEPOLIZUMAB
Name: NOM_GENERICO, dtype: object

In [22]:
def crea_base_bio(medicamento):
    _base_=procesa_medicamentos_df(base_med=biologicos,columna="NOM_GENERICO",NombreAdherencia=medicamento,Frecuencia_Mensual=1)
    _base_train_=_base_[_base_.ID_PACIENTE.isin(medicamentos_IDtrain.ID_PACIENTE)]
    _base_test_=_base_[~_base_.ID_PACIENTE.isin(medicamentos_IDtrain.ID_PACIENTE)]
    _base_train_.to_excel("Train_"+medicamento+".xlsx")
    _base_test_.to_excel("Test_"+medicamento+".xlsx")
    return  

In [23]:
crea_base_bio("OMALIZUMAB")